<a href="https://colab.research.google.com/github/ufrpe-ensino/curso-mineracao-textos/blob/master/10_IndexandoDocumentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indexando diferentes tipos de documentos

Tem uma coleção de documentos? Documentos do Word, arquivos HTML, PDFs, PDFs baseados em imagens e mais alguma coisa? Não se preocupe, o Apache [Tika](https://tika.apache.org) tem tudo para você.

# Extraindo Texto de Documentos

## Instalação
O ideal é ter um servidor dedicado ao serviço de OCR. No nosso caso, utilizaremos o próprio colab como servidor. Primeiro precisamos instalar o wrapper python para o Tika

In [ ]:
!apt-get update
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-por

In [ ]:
!pip install tika

## Baixando alguns documentos

* pdf: https://data.ct.gov/download/fxjv-82m6/application/pdf
* doc: https://pasteur.epa.gov/uploads/10.23719/1500001/LDPE_nanoclay_Highlights_.docx
* png for OCR: https://upload.wikimedia.org/wikipedia/commons/5/5f/Dr._Jekyll_and_Mr._Hyde_Text.jpg


In [ ]:
!curl -O https://upload.wikimedia.org/wikipedia/commons/5/5f/Dr._Jekyll_and_Mr._Hyde_Text.jpg

![](https://upload.wikimedia.org/wikipedia/commons/5/5f/Dr._Jekyll_and_Mr._Hyde_Text.jpg)

## Testando o tessract via linha de comando

In [ ]:
!tesseract Dr._Jekyll_and_Mr._Hyde_Text.jpg stdout

## Usando o Tika
### Inicializando o serviço

In [ ]:
import tika
import requests
from tika import parser

# Start running the tika service
tika.initVM()

## Parsing!
Existem duas maneiras de fazer isso!

**Direto da web**

```python
response = requests.get(...)
results = parser.from_buffer(response.content)
```

**De um arquivo local**

```python
results = parser.from_file(filename)
```

Observe que se você deseja fazer OCR em outro idioma, é necessário mudar um pouco as coisas. O que está abaixo para grego. Veja o que seu tesseract suporta com `tesseract --list-langs`

```python
headers = {
    "X-Tika-OCRLanguage": "por"
}
results = parser.from_buffer(response.content, headers=headers)
```

In [ ]:
!tesseract --list-langs

## Exemplo: PDF

In [ ]:
response = requests.get('https://data.ct.gov/download/fxjv-82m6/application/pdf')
results = parser.from_buffer(response)

In [ ]:
results.keys()

In [ ]:
results['status']

In [ ]:
# Only showing the first 1000 chars
results['content'][:1000]

In [ ]:
# formatando quebras de linha
print(results['content'][:1000].strip())

## Exemplo: DOC

In [ ]:
response = requests.get('https://pasteur.epa.gov/uploads/10.23719/1500001/LDPE_nanoclay_Highlights_.docx')
results = parser.from_buffer(response)
print(results['content'].strip())

## Exemplo: Imagem (OCR)

In [ ]:
response = requests.get('https://upload.wikimedia.org/wikipedia/commons/5/5f/Dr._Jekyll_and_Mr._Hyde_Text.jpg')
results = parser.from_buffer(response)
results['status']

In [ ]:
print(results['content'].strip())

## Imagem em Português
![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Livro_de_uma_sogra.djvu/page45-552px-Livro_de_uma_sogra.djvu.jpg)

In [ ]:
response = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Livro_de_uma_sogra.djvu/page45-552px-Livro_de_uma_sogra.djvu.jpg')
results = parser.from_buffer(response)
print(results['content'].strip())

**O que deu errado?**

Ajustar o idioma do tesseract!

In [ ]:
headers = {
    "X-Tika-OCRLanguage": "por"
}
response = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Livro_de_uma_sogra.djvu/page45-552px-Livro_de_uma_sogra.djvu.jpg')
results = parser.from_buffer(response, headers=headers)
print(results['content'].strip())

# Exercício

Baixe o conteúdo de alguns livros disponíveis no portal do MEC:
http://machado.mec.gov.br/obra-completa-lista

In [ ]:
docs = {'titulo':
          ['Suplicio de uma mulher',
          'Iracema',
          'Dom Casmurro'],
        'url':
          ['http://machado.mec.gov.br/obra-completa-lista/item/download/122_460dea0cb1063340b4daeb1724cd5525',
          'http://machado.mec.gov.br/obra-completa-lista/item/download/102_172f52621103b604c77a48daed4e3407',
          'http://machado.mec.gov.br/obra-completa-lista/item/download/13_7101e1a36cda79f6c97341757dcc4d04']
}

## Questão 1
Armazene os dados acima em um pandas DataFrame, e processe as urls de cada documento utilizando o Tika. Armazene os textos extraídos em uma coluna chamada `text`.

In [ ]:
import pandas as pd
df = pd.DataFrame(data=docs)

texts = []
for i, doc in df.iterrows():
  response = requests.get(doc.url)
  results = parser.from_buffer(response)
  texts.append(results['content'].strip())
df['text'] = texts
df

## Questão 2
Utilize a biblioteca hashedindex para criar um índice invertido dos documentos analisados.

Dica:


*   A função `textparser.word_tokenize` retorna uma Tupla, adicione o primeiro elemento dela ao indice (e.g., `term[0]`)
*   Otimize o índice setando o parâmetro `min_length=4` da função `textparser.word_tokenize`



In [ ]:
!pip install hashedindex
import hashedindex

index = hashedindex.HashedIndex()
for i, doc in df.iterrows():
  for term in hashedindex.textparser.word_tokenize(doc.text, min_length=4):
    index.add_term_occurrence(term[0], doc.titulo)

Quantos termos tem o seu índice?

In [ ]:
index

Quais são os termos?

In [ ]:
index.items()

Faça uma pesquisa por personagens do livro 'Dom Casmurro', por exemplo: 'capitu', 'bento', etc.

In [ ]:
index.get_documents('bento')